# Import et fonctions utilitaires

In [20]:
from PC import PC
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import time

def get_approxBN(AlgoPC:PC):
    """Fonction qui converti le graphe mixte utilisé dans l'algorithme PC en un BN

    Parameters
    ----------
    AlgoPC : PC
        l'objet associé à l'apprentissage

    Returns
    -------
    pyAgrum.BayesNet
        le BN associé
    """
    bn_approx=gum.BayesNet('bn_approx')
    for nodeID in AlgoPC.G.nodes():
        bn_approx.add(gum.LabelizedVariable(bn.variable(nodeID).name(),''))
    for (node1,node2) in AlgoPC.G.arcs():
        bn_approx.addArc(bn.variable(node1).name(),bn.variable(node2).name())
    return bn_approx

# Algorithme PC

In [21]:
#Générer un BN aléatoire
generator=gum.BNGenerator()
bn=generator.generate(n_nodes=5, n_arcs=8, n_modmax=4)
gum.generateCSV(bn,name_out="test.csv", n=20000, show_progress=False, with_labels=False)
#Fonction d'apprentissage avec PC
def learn_PC(csvFilePath="test.csv",verbose=False):
    """Approxime un BN à partir de particules données à travers un CSV donné.

    Parameters
    ----------
    csvFilePath : str, optional
        chemin vers le fichier CSV, by default "test.csv"
    verbose : bool, optional
        Permet d'afficher le graphe mixte au cours de l'apprentissage et le BN approximé à la fin avec le BN utilisé pour générer les particules, by default False

    Returns
    -------
    pyAgrum.BayesNet
        Une approximation du BN qui a généré les particules
    """
    if verbose :
        start=time.time()
    AlgoPC=PC(csvFilePath=csvFilePath)
    if verbose:
        GNonOriente=gum.MixedGraph(AlgoPC.G)
    AlgoPC.phase1()
    if verbose:
        GPhase1=gum.MixedGraph(AlgoPC.G)
    AlgoPC.phase2()
    if verbose:
        end=time.time()
        GPhase2=gum.MixedGraph(AlgoPC.G)
    bn_approx=get_approxBN(AlgoPC)
    if verbose:
        duration=end-start
        s='s' if duration>1 else ""
        print(f"Apprentissage en {duration} seconde{s}")
        print("Graphe non orienté complet | Graphe après phase 1 | Graphe après phase 2")
        gnb.sideBySide(GNonOriente,GPhase1,GPhase2)
        print("BN utilisé pour générer les particules | L'approximation du BN avec l'algorithme PC")
        gnb.sideBySide(bn,bn_approx)
    return bn_approx

#Exemple
bn_approx=learn_PC(csvFilePath="test.csv",verbose=True)


Apprentissage en 0.033509016036987305 seconde
Graphe non orienté complet | Graphe après phase 1 | Graphe après phase 2


no_name <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 <!-- 3 --> 3 3 <!-- 0->3 --> 0->3 <!-- 4 --> 4 4 <!-- 0->4 --> 0->4 <!-- 1->2 --> 1->2 <!-- 1->3 --> 1->3 <!-- 1->4 --> 1->4 <!-- 2->3 --> 2->3 <!-- 2->4 --> 2->4 <!-- 3->4 --> 3->4,no_name <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 <!-- 4 --> 4 4 <!-- 0->4 --> 0->4 <!-- 3 --> 3 3 <!-- 1->3 --> 1->3 <!-- 1->4 --> 1->4 <!-- 2->3 --> 2->3 <!-- 2->4 --> 2->4 <!-- 3->4 --> 3->4,no_name <!-- 0 --> 0 0 <!-- 4 --> 4 4 <!-- 0->4 --> 0->4 <!-- 1 --> 1 1 <!-- 1->0 --> 1->0 <!-- 3 --> 3 3 <!-- 1->3 --> 1->3 <!-- 1->4 --> 1->4 <!-- 2 --> 2 2 <!-- 2->0 --> 2->0 <!-- 2->3 --> 2->3 <!-- 2->4 --> 2->4 <!-- 3->4 --> 3->4


BN utilisé pour générer les particules | L'approximation du BN avec l'algorithme PC


## Analyse Expérimentale de PC  
### TODO : Lancer 100 apprentissages avec des CSV avec n=[1000,5000,10000,15000,20000]  
### TODO : Discuter des résultats .. critères : 
- Qualité de l'apprentissage : Structural Hamming, F-score, dist2opt, etc.  
- Rapidité de l'apprentissage 

# Algorithme PC-Stable

## Analyse Expérimentale de PC-Stable

# Conclusion

# Bonus ?